In [7]:
from preparator import Preparator 
import numpy as np 
from solver import Solver 
prep = Preparator()

from preparator import Preparator 
from logger import Logger 
import pandas as pd 
from datetime import datetime
import tqdm 
#Header
# Id (int),
# Valid (int),
# Updated (str),
# Matrix (mat),
# Size (int),
# Type (str),
# Rank (int),
# Density (flt),
# Stiffness (flt),
# Other (flt),
# Test_solver (str),
# Test (flt),
# Test_state (vec),
# Test_iterations (int),
# Test_time (flt),
# Train_solver (str),
# Preprocessing (str),
# Postprocessing (str),
# Train (flt),
# Train_state (vec),
# Train (flt), 
# Train_iterations_1 (int),
# Train_iterations_2 (int),
# Train_iterations_3 (int),
# Train_shots (int),
# Train_cost (flt),
# Train_time (flt),
# Flags (str)
# Baren (int)


In [8]:
# Prepare source dataset 

source_name = 'data/stiffness/Stiffness_size_{size}_stiffs_{stiffs}_ansatz_{ansatz}_stepsize_{stepsize}_shots_{shots}.csv'

Config = {
    'size' : 10,
    'stiffs' : [1 * 1e0, 1 * 1e1, 1 * 1e2, 1 * 1e3, 1 * 1e4, 1 * 1e5, 1 * 1e6],
    'stiff_count' : 50, 
    'shots' : 1000, 
    'stepsize' : 0.5, 
    'accepterr': [0.01,0.05,0.01], 
    'iterations_limit' : 100, 
    'baren_err' : 0.01,
    'baren_threshold': 10,
    'ansatz' : 'MA-ALT'
}

template = pd.read_csv('data/template.csv')

log = Logger (log_template = template, log_path = source_name.format(size = Config['size'], stiffs = Config['stiffs'], stepsize = Config['stepsize'], shots = Config['shots'], ansatz = Config['ansatz']))
exists = log.create_logfile()

log_file, id = log._load_logfile()


Logfile data/stiffness/Stiffness_size_10_stiffs_[1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0, 1000000.0]_ansatz_MA-ALT_stepsize_0.5_shots_1000.csv already exists


In [4]:

for stiff in Config['stiffs']: 
    for i in tqdm.tqdm(range(Config['stiff_count']), desc = f"Generating matrices with stiffness {stiff}"):
        
        id += 1
        matrix = prep.stiffnessPreparator(size = Config['size'], stiffness = stiff)
        current_datetime = datetime.now()
        data = { 
            'Id (int)' : id, 
            'Valid (int)' : 1, 
            'Updated (str)' : current_datetime.strftime("%Y-%m-%d %H:%M:%S"),
            'Matrix (mat)' : matrix,
            'Size (int)' : Config['size'],
            'Type (str)' : 'Stiff',
            'Rank (int)' : -1,
            'Density (flt)' : -1.0,
            'Stiffness (flt)' : stiff,
            'Penalty (flt)' : -1.0,
            'Other (flt)' : -1.0,
            'Test_solver (str)' : '-',
            'Test (flt)' : -1.0,
            'Test_state (vec)' : [],
            'Test_iterations (int)' : -1,
            'Test_time (flt)' : -1.0,
            'Train_solver (str)' : '-',
            'Preprocessing (str)' : '-',
            'Postprocessing (str)' : '-',
            'Train (flt)' : -1.0,
            'Train_state (vec)' : [],
            'Train_iterations (int)' : -1, 
            'Train_iterations_1 (int)' : -1,
            'Train_iterations_2 (int)' : -1,
            'Train_iterations_3 (int)' : -1,
            'Train_shots (int)': -1, 
            'Train_cost (flt)' : -1.0,
            'Train_time (flt)' : -1.0, 
            'Flags (str)' : '',
            'Baren (int)' : -1
        }
        
        log.log(data)

    
display(log.read(1))

Generating matrices with stiffness 1.0:   0%|          | 0/50 [00:00<?, ?it/s]

Generating matrices with stiffness 100000.0:  84%|████████▍ | 42/50 [00:03<00:00, 11.09it/s]


KeyboardInterrupt: 

In [3]:
import pennylane as qml 
testdev = qml.device ('default.qubit', wires = Config['size'], shots = Config['shots'])


In [7]:

def test (ids : np.ndarray, solver : str = 'bruteforce', log_data : bool = False, progressbar : bool = False ):

    ''' 
    ids : np.ndarray 
        a list of ids to compute 
    solver : str 
        solver to compute with 
    ''' 
    
    tester = Solver(dev = testdev, ansatz = '')

    if (progressbar):
        id_range = tqdm(ids, desc = f"Testing matrices with ids {min(ids)} <--> {max(ids)}") 
    else: 
        id_range = ids
    processed_data = []
    for id in id_range:

        data = log.read(id)

        print(data)
        def test_solver(Q : np.ndarray, solver : str = 'bruteforce'): 
            ''' 
            tester wrapped
            '''

            match solver: 
                case 'bruteforce':
                    J, h = prep.isingForm(Q)
                    return tester.checkIsing(J, h)
                case 'GW':
                    pass
                case 'Annealing': 
                    pass
            
        Q = data['Matrix (mat)']

        tested_data = data.copy()
        
        test, test_state, test_time = test_solver(Q)
        current_datetime = datetime.now()

        tested_data['Test (flt)'] = test 
        tested_data['Test_state (vec)'] = test_state[0].numpy()
        tested_data['Test_time (flt)'] = test_time
        tested_data['Test_solver (flt)'] = solver
        tested_data['Updated (str)'] = current_datetime.strftime("%Y-%m-%d %H:%M:%S")

        processed_data.append(tested_data)

        if (log_data): 
            log.log(tested_data, byid = True, Id = id)
    return processed_data


In [ ]:
# test(range(1,21), solver = 'bruteforce')

Columns in data/stiffness/Stiffness_size_10_stiffs_[1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0, 1000000.0]_stepsize_0.25_shots_1000.csv are broken. Expected Index(['Id (int)', 'Valid (int)', 'Updated (str)', 'Matrix (mat)',
       'Size (int)', 'Type (str)', 'Rank (int)', 'Density (flt)',
       'Stiffness (flt)', 'Penalty (flt)', 'Other (flt)', 'Test_solver (str)',
       'Test (flt)', 'Test_state (vec)', 'Test_iterations (int)',
       'Test_time (flt)', 'Train_solver (str)', 'Preprocessing (str)',
       'Postprocessing (str)', 'Train (flt)', 'Train_state (vec)',
       'Train_iterations (int)', 'Train_iterations_1 (int)',
       'Train_iterations_2 (int)', 'Train_iterations_3 (int)',
       'Train_cost (flt)', 'Train_shots (int)', 'Train_time (flt)',
       'Flags (str)', 'Baren (int)'],
      dtype='object') and got Index(['Id (int)', 'Valid (int)', 'Updated (str)', 'Matrix (mat)',
       'Size (int)', 'Type (str)', 'Rank (int)', 'Density (flt)',
       'Stiffness (flt)', 'Other 

KeyboardInterrupt: 

In [9]:


ids = range (701,1051)
cores = 35
poolsize = len(ids)//cores

from multiprocessing import Pool
from functools import partial

if (__name__ == '__main__') and (len(ids) % poolsize == 0): 
    
    solvers = [] 

    ids = np.array(ids) 
    index = 0
    pool_ids = ids.reshape(len(ids) // poolsize, poolsize)
    display(pool_ids)
    
    args = []
    for i in range (len(pool_ids)):
        if (Config['size'] > 12):
            args.append((pool_ids[i], 'GW'))
        else: 
            args.append((pool_ids[i], 'bruteforce'))
    #print(args)
    with Pool(cores) as pool:
        processed_data = pool.starmap(test, args)


for i in range(cores):
    for data in processed_data[i]:
        log.log(data = data, byid = True, Id = data['Id (int)'])

# If the error occured you still can try to get processed_data 



array([[ 701,  702,  703,  704,  705,  706,  707,  708,  709,  710],
       [ 711,  712,  713,  714,  715,  716,  717,  718,  719,  720],
       [ 721,  722,  723,  724,  725,  726,  727,  728,  729,  730],
       [ 731,  732,  733,  734,  735,  736,  737,  738,  739,  740],
       [ 741,  742,  743,  744,  745,  746,  747,  748,  749,  750],
       [ 751,  752,  753,  754,  755,  756,  757,  758,  759,  760],
       [ 761,  762,  763,  764,  765,  766,  767,  768,  769,  770],
       [ 771,  772,  773,  774,  775,  776,  777,  778,  779,  780],
       [ 781,  782,  783,  784,  785,  786,  787,  788,  789,  790],
       [ 791,  792,  793,  794,  795,  796,  797,  798,  799,  800],
       [ 801,  802,  803,  804,  805,  806,  807,  808,  809,  810],
       [ 811,  812,  813,  814,  815,  816,  817,  818,  819,  820],
       [ 821,  822,  823,  824,  825,  826,  827,  828,  829,  830],
       [ 831,  832,  833,  834,  835,  836,  837,  838,  839,  840],
       [ 841,  842,  843,  844,  8

{'Id (int)': 781, 'Valid (int)': 1, 'Updated (str)': '2025-08-24 04:13:29', 'Matrix (mat)': array([[-0.16408459, -0.25281417, -0.05315995, -0.19082207, -0.15961414,
         0.0638192 ,  0.12202632, -0.17734082,  0.21064234, -0.21382121],
       [-0.25281417, -0.72951882, -0.1413846 ,  0.16622933, -0.44885979,
         0.13130659,  0.1102221 , -0.11777096, -0.01406754, -0.09809205],
       [-0.05315995, -0.1413846 , -0.3746252 , -0.19990574, -0.09662612,
        -0.20252096,  0.16695862, -0.25028537,  0.14241271, -0.00779399],
       [-0.19082207,  0.16622933, -0.19990574, -0.04301711,  0.10891294,
         0.25370874, -0.21510086, -0.00417482, -0.00502361, -0.12855388],
       [-0.15961414, -0.44885979, -0.09662612,  0.10891294, -0.48728127,
         0.13468082,  0.1747261 , -0.01708025, -0.10394901,  0.13675421],
       [ 0.0638192 ,  0.13130659, -0.20252096,  0.25370874,  0.13468082,
         0.01422015, -0.2321558 ,  0.08865985, -0.16081353,  0.19826376],
       [ 0.12202632,  0.11

In [9]:

def train (data : list, trainer : Solver, stepsize: float = 0.1, depth : int = 1, log_data = False): 
    ''' 
    ids : np.ndarray 
        array of ids to train
    solver : str 
        solver with which will be trained 
    preprocessing : str 
    postprocessing : str 
    '''

    #print(f'training ids {ids}')
    #print(data)
    processed_data = []
    for line in data:
        #print(type(line))
        Q = line['Matrix (mat)']

        trained_data = line.copy()
        
        train, train_bitstring, train_state, train_iterations, train_itererr, train_time, baren_flag, baren_index = trainer.solve(Q = Q, depth = depth, sol = line['Test (flt)'], stepsize = stepsize, log = False)
        print(f'Trained id = {line['Id (int)']} with train {train}')
        trained_data['Train (flt)'] = train
        trained_data['Train_iterations (int)'] = train_iterations
        trained_data['Train_state (vec)'] = train_bitstring
        for i in range(1, 4): trained_data[f'Train_iterations_{i} (int)'] = train_itererr[i - 1]
        trained_data['Train_time (flt)'] = train_time
        
        current_datetime = datetime.now()
        trained_data['Updated (str)'] = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
        
        if (baren_flag == True):
            trained_data['Flags (str)'] = f'Baren_{baren_index}'
        trained_data['Train_shots (int)'] = trainer.dev.shots.total_shots
        trained_data['Train_solver (str)'] = trainer.ansatz
        trained_data['Preprocessing (str)'] = trainer.preprocessing 
        trained_data['Postprocessing (str)'] = trainer.postprocessing
        if (log_data):
            log.log(trained_data, byid = True, Id = id)
        processed_data.append(trained_data)
    return processed_data

In [ ]:
from multiprocessing import Pool
import pandas as pd 
import pennylane as qml 

cores = 10

solvers = [] 

depth = 3

template = pd.read_csv('data/template.csv')

for i in range (cores):
    dev = qml.device ('lightning.qubit', wires = Config['size'], shots = Config['shots'])
    solvers.append(Solver(dev = dev, ansatz = 'MA-PIA', gate = 'X', iterations_limit = Config['iterations_limit'], accepterr = Config['accepterr'], baren_check = True, baren_rerr = Config['baren_err']))


log = Logger(log_template = template, log_path = source_name.format(size = Config['size'], stiffs = Config['stiffs'], stepsize = Config['stepsize'], shots = Config['shots'], ansatz = Config['ansatz']))
train_dataset, _ = log._load_logfile()
#print(train_dataset)

ids = np.array(range(1, 351))
ids = ids.reshape(cores, len(ids) // cores) 

data = [[log.read(id) for id in subset] for subset in ids]

#print(data)
if (__name__ == '__main__') and (len(train_dataset) % cores):
    
    args = [(data[i], solvers[i], Config['stepsize'], depth) for i in range (cores)]
    #print(train_dataset)
    #print(args)
    with Pool(cores) as pool:
        processed_data = pool.starmap(train, args)


for i in range(cores):
    for data in processed_data[i]:
        log.log(data = data, byid = True, Id = data['Id (int)'])




Trained id = 211 with train -1.422599269564897
Trained id = 212 with train -6.597019946215017
Trained id = 246 with train -1.9377378714684568
Trained id = 316 with train -2.2692105709378056
Trained id = 247 with train -7.305089670157082
Trained id = 281 with train -3.1944263325385176
Trained id = 317 with train -3.2730241792376056
Trained id = 1 with train -3.401374558752802
Trained id = 176 with train -9.436743353724319
Trained id = 213 with train -8.200487794721216
Trained id = 177 with train -6.865917429123078
Trained id = 2 with train -4.239555827406414
Trained id = 214 with train -7.460306615651748
Trained id = 282 with train -4.627222706742026
Trained id = 36 with train -3.408217458885459
Trained id = 141 with train -5.19711132276633
Trained id = 178 with train -3.717824628517678
Trained id = 283 with train -4.900046094042384
Trained id = 215 with train -1.9697570864029323
Trained id = 142 with train -5.986030846909557
Trained id = 3 with train -2.826748766810756
Trained id = 284

In [ ]:
display(processed_data[3]['Id (int)'])
display(processed_data[3]['Train (flt)'])

arr = [1,2,3,4,5,6,7,8,9,10]
arr = np.array(arr)
arr = arr.reshape(len(arr) // 2, 2)

print(arr)
train(range(1,21), trainer = solvers[0], progressbar = True)
